## Trajectory analysis template
General geometric trajectory analysis from a ```.mymridon``` experiment file to be saved as a csv, which can be further analyzed in the optional part or exported into other software such as RStudio. Operations like done here are typically heavy and python benefits from the effeciency of the underlying ```myrmidon``` library to a big extent, especially in combination with the ```numpy``` math library.
This notebook is a walk trough a sample usage of the following:
* the `py-fort-myrmidon` library ([Documentation](https://formicidae-tracker.github.io/myrmidon/latest/))
* numpy quick start [tutorial](https://numpy.org/devdocs/user/quickstart.html)

In [ ]:
import py_fort_myrmidon as fm
import numpy as np  # Fundamental math library in python. Here used only for convience: to save the csv.
from datetime import datetime, timedelta  # For convenient handling of time and date
import numpy as np  # Basic math library
import pandas as pd  # Used to create a dataframe, similar to the structure used in R
import matplotlib.pyplot as plt  # Optional: for plorring

# Optional: makes plots interactive:
# %matplotlib widget

# %load_ext blackcellmagic

In [ ]:
myrmidonFilepath = "dataset/example.myrmidon"
experiment = fm.Experiment.Open(myrmidonFilepath)
t_start = fm.Time.Parse("2021-02-18T00:00:00.000Z")
t_end = fm.Time.Parse("2021-02-18T01:00:00.000Z")
# alternatively we can get the whole wxperiment range
# di = fm.Query.GetDataInformations(experiment)
# t_start, t_end = di.Start, di.End

The following is an iterator for `fort-myrmidon` time range over days in a period. See Ant metadata template for explanation.

In [ ]:
def fm_time_range(
    start: fm.Time,
    end: fm.Time,
    *,
    increment=24 * fm.Duration.Hour,
):
    """Slice a time range is sub-time range 'à la' range()

    Args:
        start (fm.Time): the start time to consider
        end (fm.Time): the end time to consider
        increment (fm.Duration): the time increment to slice the whole time range

    Yields:
        Tuple[fm.Time,fm.Time]: start and end time of the sub range that span at most 'increment'
    """
    while start < end:
        last = start
        start = min(start.Add(increment), end)
        yield last, start

Calculate the mean speed in pixels/second of each ant during the period (```t_start```, ```t_end```). This can serve as measure for the activity of the individual during that period. If the ```maximumGap``` is set to a period larger than (```t_start```, ```t_end```), then every ```ant_id``` will have at most one trajectory which makes the analysis easier in this case (we do not need to accumulate every speed and then calculate the final means/stds). The meaning of the positions in a trajectory are described [here](https://formicidae-tracker.github.io/myrmidon/latest/api/python/queries_matchers.html#py_fort_myrmidon.AntTrajectory.Positions). Another option would be to calculate the angular speed.

In [ ]:
df_trajectory_stats = pd.DataFrame(index=experiment.Ants)
df_trajectory_stats["speed_mean"] = np.nan
df_trajectory_stats["speed_std"] = np.nan
for t_begin, t_last in fm_time_range(t_start, t_end):
    print(f"computing trajectories in {t_begin,t_last}")
    trajectories = fm.Query.ComputeAntTrajectories(
        experiment=experiment,
        start=t_begin,
        end=t_last,
        maximumGap=1000 * fm.Duration.Hour,
    )
    for t in trajectories:
        dxdy = np.diff(
            t.Positions[:, 1:3], axis=0
        )  # x-y differerence between detections (2d vector)
        ds = np.linalg.norm(dxdy, axis=1)  # displacement between detections (1d vector)
        dt = np.diff(t.Positions[:, 0])  # Time interval between detections (1d vector)
        speed = np.divide(
            ds, dt, where=dt > 0
        )  # [pixels / second]  (zero dt gaps are quite rare but can occur)
        df_trajectory_stats.loc[t.Ant, "speed_mean"] = np.mean(speed)
        df_trajectory_stats.loc[t.Ant, "speed_std"] = np.std(speed)

Save dataframe to a csv and optionally show and plot the results.

In [ ]:
f_name = "ant_trajectory_stats_{}_{}_{}.csv".format(experiment.Name, t_start, t_end)
df_trajectory_stats.to_csv(f_name, index_label="ant_id")

In [ ]:
df_trajectory_stats.pivot_table(
    values="speed_mean", index=df_trajectory_stats.index
).plot(kind="bar", figsize=(8, 3), ylabel="[px/sec]")
df_trajectory_stats

## Calculate displacement in specific phase

In [ ]:
def calculate_displacement(trajectory):
    """Function to create a dataframe containing displacment per detection in a trajectory

    Args:
        trajectory (fort-myrmidon Ant trajectory): A trajectory object obtained using the fort-myrmidon API

    Returns:
        pd.DataFrame: Outputs a pandas dataframe with the time stamps as index, AntID, X and Y coordinates, displacment and time difference for the timestamp (relative to the preceding timestamp)
    """
    # calculate displacement and time difference between detections
    dxdy = np.diff(
        trajectory.Positions[:, 1:3], axis=0
    )  # x-y differerence between detections (2d vector)
    ds = np.linalg.norm(dxdy, axis=1)  # displacement between detections (1d vector)
    ds = np.insert(ds, 0, 0)  # Add a zero displacement for the first detection
    dt = np.diff(
        trajectory.Positions[:, 0]
    )  # Time interval between detections (1d vector)
    # Get time information
    dt = np.insert(dt, 0, 0)  # Add a zero time difference for the first detection
    trajectory_start = trajectory.Start.ToDateTime()  # Get starting time of trajectory
    trajectory_time = trajectory.Positions[
        :, 0
    ]  # get time difference of detections from starting time
    trajectory_time = pd.to_timedelta(
        trajectory_time, unit="s"
    )  # Convert time difference to timedelta
    time_stamps = trajectory_start + trajectory_time  # Get time stamps
    # time_stamps = time_stamps.delete(
    #     0
    # )  # Remove first time stamp, since we don't have displacement for it
    # Create dataframe
    disp_df = pd.DataFrame(index=time_stamps, data=ds, columns=["displacement"])
    disp_df["time_diff"] = dt
    disp_df["AntID"] = trajectory.Ant  # Add Ant ID
    disp_df["X"] = trajectory.Positions[:, 1]  # Add X position
    disp_df["Y"] = trajectory.Positions[:, 2]  # Add Y position
    disp_df["Space"] = trajectory.Space  # Add Space ID
    disp_df = disp_df[["AntID", "Space", "X", "Y", "displacement", "time_diff"]]
    return disp_df

In [ ]:
def obtain_displacement_df(exp, start_time, end_time, phase, *time_resolution):
    """Output a csv the displacement (or mean displacement) of each ant in the experiment between t_start and t_end.

    Args:
        exp (fm.Experiment): the experiment object
        t_start (fm.Time): the start time
        t_end (fm.Time): the end time
        phase (fm.Phase): the phase to consider
        time_resolution (string): the time resolution to consider, given in the format used by pandas grouper
    """
    t_start = fm.Time(start_time)
    t_end = fm.Time(end_time)
    freq = "".join(time_resolution)
    trajectories = fm.Query.ComputeAntTrajectories(
        experiment=exp, start=t_start, end=t_end, maximumGap=1000 * fm.Duration.Hour
    )
    disp_df_list = [calculate_displacement(t) for t in trajectories]
    disp_df = pd.concat(disp_df_list)
    disp_df.index.name = "time"
    # Since thee are instancs where the ant may not be detected for a given duration, we filter these out.
    # We keep the filter to a value of 1s. This ensures that if there is a gap in dtection greater than a second,
    # that specific value is not included in the calculation
    disp_df = disp_df[
        disp_df["time_diff"] < 1
    ]  # Filter out rows where time_diff is greater than 1 second
    day_exp = start_time.strftime("%Y%m%d")
    hr_strt = start_time.strftime("%H%M")
    hr_end = end_time.strftime("%H%M")
    if time_resolution:
        disp_df = (
            disp_df.groupby([pd.Grouper(freq=freq), "AntID", "Space"])
            .agg(
                {"X": "mean", "Y": "mean", "displacement": "mean", "time_diff": "mean"}
            )
            .reset_index()
        )
        df_fn = f"{'MeanDisplacement_'}{freq}{'_'}{exp.Name}{'_'}{phase}{'_'}{day_exp}{'_'}{hr_strt}{'-'}{hr_end}{'.csv'}"
        disp_df.to_csv(df_fn, index=False)
    else:
        df_fn = f"{'Displacement_'}{exp.Name}{'_'}{phase}{'_'}{day_exp}{'_'}{hr_strt}{'-'}{hr_end}{'.csv'}"
        disp_df.reset_index().to_csv(df_fn, index=False)

In [ ]:
phase_list = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]
time_resolution = "1s"
# phase_list = phase_list[1:6]
# phase_list = [phase_name + 'Plus2' for phase_name in phase_list]

### Colony Cfel 42

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
phase_starts_exp = [
    datetime(2022, 5, 1, 15, 54).astimezone(tz=None),
    datetime(2022, 5, 2, 16, 3).astimezone(tz=None),
    datetime(2022, 5, 3, 15, 53).astimezone(tz=None),
    datetime(2022, 5, 4, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 5, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 6, 15, 55).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2022, 5, 2, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 7, 9, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel42_disp = [
    obtain_displacement_df(exp, start, end, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel1

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
phase_starts_exp = [
    datetime(2022, 6, 4, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 5, 14, 57).astimezone(tz=None),
    datetime(2022, 6, 6, 14, 30).astimezone(tz=None),
    datetime(2022, 6, 7, 14, 49).astimezone(tz=None),
    datetime(2022, 6, 8, 14, 43).astimezone(tz=None),
    datetime(2022, 6, 9, 15, 5).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2022, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 10, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel1_disp = [
    obtain_displacement_df(exp, start, end, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel54

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
phase_starts_exp = [
    datetime(2022, 6, 19, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 20, 14, 35).astimezone(tz=None),
    datetime(2022, 6, 21, 14, 21).astimezone(tz=None),
    datetime(2022, 6, 22, 14, 28).astimezone(tz=None),
    datetime(2022, 6, 23, 14, 14).astimezone(tz=None),
    datetime(2022, 6, 24, 14, 31).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2022, 6, 20, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 24, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 25, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel54_disp = [
    obtain_displacement_df(exp, start, end, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel55

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-3/InfectionExp_Cfel55/InfectionExpCol55.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
phase_starts_exp = [
    datetime(2023, 4, 18, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 20, 15, 45).astimezone(tz=None),
    datetime(2023, 4, 21, 14, 48).astimezone(tz=None),
    datetime(2023, 4, 22, 14, 17).astimezone(tz=None),
    datetime(2023, 4, 23, 14, 0).astimezone(tz=None),
    datetime(2023, 4, 24, 14, 54).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2023, 4, 20, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 21, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 22, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 23, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel55_disp = [
    obtain_displacement_df(exp, start, end, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel13

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-3/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
phase_starts_exp = [
    datetime(2023, 4, 23, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 24, 15, 29).astimezone(tz=None),
    datetime(2023, 4, 25, 14, 19).astimezone(tz=None),
    datetime(2023, 4, 26, 15, 3).astimezone(tz=None),
    datetime(2023, 4, 27, 16, 43).astimezone(tz=None),
    datetime(2023, 4, 28, 14, 27).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 26, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 27, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 28, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 29, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
# Run only if you need to get displacement for 2 hours after estart of each treatment phase
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel13_disp = [
    obtain_displacement_df(exp, start, end, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]

### Colony Cfel64

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-1/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
phase_starts_exp = [
    datetime(2023, 5, 31, 15, 5).astimezone(tz=None),
    datetime(2023, 6, 1, 15, 51).astimezone(tz=None),
    datetime(2023, 6, 2, 14, 44).astimezone(tz=None),
    datetime(2023, 6, 3, 14, 50).astimezone(tz=None),
    datetime(2023, 6, 4, 14, 43).astimezone(tz=None),
    datetime(2023, 6, 5, 14, 52).astimezone(tz=None),
]
phase_starts_post = [
    datetime(2023, 6, 1, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 2, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 3, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 4, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 6, 8, 0).astimezone(tz=None),
]
phase_starts = phase_starts_exp + phase_starts_post
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
phase_starts = phase_starts[1:6]
phase_starts = [(start_time + timedelta(hours=2)) for start_time in phase_starts]
phase_ends = [(start_time + timedelta(hours=6)) for start_time in phase_starts]

In [ ]:
cfel64_disp = [
    obtain_displacement_df(exp, start, end, phase, time_resolution)
    for start, end, phase in zip(phase_starts, phase_ends, phase_list)
]